In [66]:
import numpy as np
import random
import matplotlib.pyplot as plt
from random import uniform
from collections import Counter
from tqdm import tqdm

In [67]:
# Parameters
N = 50 # Number of nodes
num_simulations = 30 # Number of simulations per setting
T = 50 # Number of time steps
initial_fitness = 100  # Initial fitness for all nodes
info_prop = .61 # 3 degrees of separation 1st degree
num_inter_epoch = N

# Prisioner's dilemma
coop_gain = 0.2  # Gain when both cooperate
def_loss = -0.05 # Loss when both defect
suck_payoff = -1  # Loss when one cooperates and the other defects for the cooperator
temptation = 0.6  # Gain when one cooperates and the other defects for the defector

coop_coop_gain = 2*coop_gain
coop_def_gain = suck_payoff + temptation
def_def_gain = 2*def_loss

In [68]:
# Names
COOPERATOR = 1
DISCRIMINATOR = 0
DEFECTOR = -1

COOPERATE = True
DEFECT = False

GOOD = True
BAD = False

As we are using boolean values both for the reputation/standing of the nodes and the cooperation/defection actions, with XNOR, we get the next table:

| Result | G | B  |
| ------ | - | -- |
| C      | G | B  |
| D      | B | G  |

Which corresponds to the "Stern Judging" moral system of indirect reciprocity

## Well-mixed

In [69]:
# Simulation results initialization
sim_total_fitness = np.empty(T, dtype=np.float64)
sim_cooperative_act = np.empty(T, dtype=np.float64)
sim_cooperators = np.empty(T, dtype=np.float64)
sim_defectors = np.empty(T, dtype=np.float64)
sim_discriminators = np.empty(T, dtype=np.float64)

# Aggregate results initialization
res_total_fitness = np.zeros(T, dtype=np.float64)
res_coop_actions = np.zeros(T, dtype=np.float64)
res_cooperators = np.zeros(T, dtype=np.float64)
res_defectors = np.zeros(T, dtype=np.float64)
res_discriminators = np.zeros(T, dtype=np.float64)

In [70]:
def init(f_coop, f_def, f_disc, p_inf):
    global nodes, fitness, standing, reputation, perfect_inf

    # Initialization of variables
    nodes = np.random.choice([COOPERATOR, DEFECTOR, DISCRIMINATOR], size=N, p=[f_coop, f_def, f_disc])
    fitness = np.full(N, initial_fitness, dtype=np.float64)
    standing = np.ones(N, dtype=np.bool_)
    reputation = np.ones((N, N), dtype=np.bool_)

    perfect_inf = p_inf

In [71]:
def decision(node_id, other_id):
    node_t = nodes[node_id]
    other_t = nodes[other_id]
    
    if node_t == COOPERATOR:
        return COOPERATE
    elif other_t == DEFECTOR:
        return DEFECT
    
    # Discriminator
    if perfect_inf:
        rep_other = standing[other_id]
    else:
        rep_other = reputation[node_id, other_id]

    return rep_other

In [72]:
def update_rep(observer, node1, node2, dec1, dec2, prob_obs):
    # Update with the given probability
    if uniform(0, 1) > prob_obs:
        return
    
    good1 = reputation[observer, node1]
    good2 = reputation[observer, node2]

    # XNOR -> Stern judging
    reputation[observer, node1] = not (dec1 ^ good2)
    reputation[observer, node2] = not (dec2 ^ good1)

In [73]:
def interaction(node1, node2):
    dec1 = decision(node1, node2)
    dec2 = decision(node2, node1)

    ret = 0
    # Fitness
    ## COOP-COOP
    if dec1 and dec2:
        # Fitness
        fitness[node1] += coop_gain
        fitness[node2] += coop_gain
        ret = 1
    ## COOP-DEF
    elif dec1 and (not dec2):
        fitness[node1] += suck_payoff
        fitness[node2] += temptation
    ## DEF-COOP
    elif dec2:
        fitness[node1] += temptation
        fitness[node2] += suck_payoff
    ## DEF-DEF
    else:
        fitness[node1] += def_loss
        fitness[node2] += def_loss
    
    # Standing
    good1 = standing[node1]
    good2 = standing[node2]
    ## XNOR -> stern judging
    standing[node1] = not (dec1 ^ good2)
    standing[node2] = not (dec2 ^ good1)

    # Reputation
    ## (Nodes in the interaction always observe the interaction)
    good1_1 = reputation[node1, node1]
    good1_2 = reputation[node1, node2]
    good2_1 = reputation[node2, node1]
    good2_2 = reputation[node2, node2]
    ## XNOR -> stern judging
    new_rep1_1 = not (dec1 ^ good1_2)
    new_rep1_2 = not (dec2 ^ good1_1)
    new_rep2_1 = not (dec1 ^ good2_2)
    new_rep2_2 = not (dec2 ^ good2_1)
    ## Apply update to all nodes with the given probability
    update_observer_rep = lambda observer: update_rep(observer, node1, node2, dec1, dec2, info_prop)
    for i in range(N):
        update_observer_rep(i)
    ## Update the interacting nodes' reputation
    reputation[node1, node1] = new_rep1_1
    reputation[node1, node2] = new_rep1_2
    reputation[node2, node1] = new_rep2_1
    reputation[node2, node2] = new_rep2_2
    
    # Returns 1 if it was a cooperation and 0 if it was a defection
    return ret

In [74]:
def run_epoch(id):
    coop_act = 0
    for i in range(num_inter_epoch):
        node1, node2 = np.random.choice(N, 2, False)
        coop_act += interaction(node1, node2)
    
    # Calculate epoch metrics
    sim_total_fitness[id] = np.sum(fitness)
    sim_cooperative_act[id] = coop_act
    node_types = Counter(nodes)
    sim_cooperators[id] = node_types[COOPERATOR]
    sim_defectors[id] = node_types[DEFECTOR]
    sim_discriminators[id] = node_types[DISCRIMINATOR]

In [75]:
def run_sim(f_coop, f_def, f_disc, perfect_info):
    global res_total_fitness, res_coop_actions, res_cooperators, res_defectors, res_discriminators

    init(f_coop, f_def, f_disc, perfect_info)
    
    for epoch in range(T):
        run_epoch(epoch)
    
    # Add to result aggregates
    res_total_fitness = np.add(res_total_fitness, sim_total_fitness)
    res_coop_actions = np.add(res_coop_actions, sim_cooperative_act)
    res_cooperators = np.add(res_cooperators, sim_cooperators)
    res_defectors = np.add(res_defectors, sim_defectors)
    res_discriminators = np.add(res_discriminators, sim_discriminators)

In [76]:
reputation[1, 1] or True

np.True_

### Simulation run and plotting of results

In [77]:
for _ in tqdm(range(num_simulations)):
    run_sim(0.33, 0.33, 0.34, True)

100%|██████████| 30/30 [00:08<00:00,  3.58it/s]
